### Librerias

In [1]:
import pymysql 
from sqlalchemy import create_engine
import pandas as pd


### Credenciales

In [2]:
username = "admin_cloud"
password = "grupo2DS"
host = "database-1.cf0hxwxsba9n.us-east-2.rds.amazonaws.com" 
port = "3306"

### Conexion BD

In [3]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [4]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.28'}


### Creacion DB

In [6]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE users_web'''
cursor.execute(create_db)

# Podemos eliminar la BD
#drop_db = '''DROP DATABASE users_web'''
#cursor.execute(drop_db)

# La volvemos a crear
#cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [7]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'},
 {'Database': 'users_web'}]

### Seleccionar BD

In [8]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE users_web'''
cursor.execute(use_db)

0

### Creacion de tablas

In [9]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''

# crear una tabla que se llame users_web

create_table = '''
CREATE TABLE usuarios_web (
    Date DATE,
    Users INT,
    primary key (Date)
)
'''
cursor.execute(create_table)

0

In [10]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_users_web': 'usuarios_web'}]

In [11]:
# Para borrar las tablas

# drop_table = '''DROP TABLE usuarios_web'''
# cursor.execute(drop_table)

### Insertar datos de un CSV

In [26]:
# leemos los datos
df = pd.read_csv('data/users_web.csv')
df = df[:-30]
df.head()

,Date,Users
0,1/1/2020,13
1,2/1/2020,17
2,3/1/2020,13
3,4/1/2020,4
4,5/1/2020,10


In [27]:
# Cambiamos el formato de la fecha de 00/00/0000 a 0000/00/00
df['Date'] = pd.to_datetime(df['Date'])
#df = df.set_index('Date')
df

C:\Users\sesco\AppData\Local\Temp\ipykernel_6220\1140421151.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])


,Date,Users
0,2020-01-01,13
1,2020-02-01,17
2,2020-03-01,13
3,2020-04-01,4
4,2020-05-01,10
...,...,...
816,2022-03-27,31
817,2022-03-28,66
818,2022-03-29,66
819,2022-03-30,71


In [28]:
print(username)
print(password)
print(host)
print('users_web')

admin_cloud
grupo2DS
database-1.cf0hxwxsba9n.us-east-2.rds.amazonaws.com
users_web


In [29]:
# create sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'users_web'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [30]:
# insertamos todo el dataframe
df.to_sql(name='usuarios_web', con=engine, if_exists= 'append', index=False)

821

In [31]:
db.commit()

In [32]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM usuarios_web'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'Date': datetime.date(2020, 1, 1), 'Users': 13},
 {'Date': datetime.date(2020, 1, 2), 'Users': 46},
 {'Date': datetime.date(2020, 1, 3), 'Users': 27},
 {'Date': datetime.date(2020, 1, 4), 'Users': 10},
 {'Date': datetime.date(2020, 1, 5), 'Users': 9},
 {'Date': datetime.date(2020, 1, 6), 'Users': 28},
 {'Date': datetime.date(2020, 1, 7), 'Users': 45},
 {'Date': datetime.date(2020, 1, 8), 'Users': 38},
 {'Date': datetime.date(2020, 1, 9), 'Users': 58},
 {'Date': datetime.date(2020, 1, 10), 'Users': 32},
 {'Date': datetime.date(2020, 1, 11), 'Users': 41},
 {'Date': datetime.date(2020, 1, 12), 'Users': 27},
 {'Date': datetime.date(2020, 1, 13), 'Users': 27},
 {'Date': datetime.date(2020, 1, 14), 'Users': 33},
 {'Date': datetime.date(2020, 1, 15), 'Users': 16},
 {'Date': datetime.date(2020, 1, 16), 'Users': 14},
 {'Date': datetime.date(2020, 1, 17), 'Users': 14},
 {'Date': datetime.date(2020, 1, 18), 'Users': 18},
 {'Date': datetime.date(2020, 1, 19), 'Users': 25},
 {'Date': datetime.dat

In [33]:
mi_tabla[820]

{'Date': datetime.date(2022, 12, 3), 'Users': 32}

In [34]:
pd.DataFrame(mi_tabla)

,Date,Users
0,2020-01-01,13
1,2020-01-02,46
2,2020-01-03,27
3,2020-01-04,10
4,2020-01-05,9
...,...,...
816,2022-11-02,46
817,2022-11-03,28
818,2022-12-01,46
819,2022-12-02,49


### Cerrar la conexión cuando acabemos

In [35]:
# Cerrar conexión mysql
db.close()

In [36]:
# Cerrar conexión SQLalchemy
conn = engine.connect()
conn.close()
conn = engine.dispose()
